# Import modules

In [1]:
%%time

import datetime
import os, glob, sys

import warnings
warnings.filterwarnings('ignore', '.*invalid value encountered in true_divide.*', )

import pickle



import numpy as np
import xarray as xr
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from matplotlib import cm
import cartopy.crs       as ccrs
import cartopy

from scipy.interpolate import UnivariateSpline
from sklearn.metrics import r2_score, mean_squared_error, explained_variance_score
from scipy.stats import pearsonr, spearmanr

mpl.rcParams['savefig.dpi'] = 300

CPU times: user 1.23 s, sys: 168 ms, total: 1.4 s
Wall time: 1.42 s


# Starters

In [2]:
%%time
print(datetime.datetime.now())

dirout = 'dimred-220513-predictors-variances/'
if not os.path.isdir(dirout) : os.mkdir(dirout)

2022-11-08 11:33:13.094226
CPU times: user 444 µs, sys: 62 µs, total: 506 µs
Wall time: 304 µs


In [3]:
%%time
print(datetime.datetime.now())
sys.stdout.echo = open(dirout+'stdout.txt', 'w')
sys.stderr.echo = open(dirout+'stderr.txt', 'w')

2022-11-08 11:33:13.102735
CPU times: user 576 µs, sys: 934 µs, total: 1.51 ms
Wall time: 25.7 ms


# Input parameters

In [4]:
%%time
print(datetime.datetime.now())

dirshared = '/mnt/reef-ns1002k/daco/MY_JUPYTER_NOTEBOOKS/DIM-REG/SHARED-DATAS/'
dircmip6 = '/mnt/reef-ns1002k/daco/MY_JUPYTER_NOTEBOOKS/DIM-RED/CMIP6-DATAS-LINKS/'

2022-11-08 11:33:13.133849
CPU times: user 362 µs, sys: 52 µs, total: 414 µs
Wall time: 282 µs


# Define functions

In [5]:
def p_smooth_spline(t,x):
    spline = UnivariateSpline(t, x, k=3, bbox=[np.min(t), np.max(t)])
    y = spline(t)
    return(y)
#

def dtrd_dseas(zwraw, zwtrd): 
    zwdtrd = zwraw - zwtrd
    zw = []
    for mmm in np.arange(12): zw.append(np.nanmean(zwdtrd[mmm::12]))
    zwseas = []
    for yyy in np.arange(len(zwraw)/12): zwseas.extend(zw)
    zwseas = np.array(zwseas)
    return zwraw - zwtrd - zwseas, zwseas+zwtrd, zwseas
#



# Variances of the detrended and deseasonalized predictors

## Inputs

In [6]:
%%time
print(datetime.datetime.now())

# model_list = ['NorESM2-LM', 'MPI-ESM1-2-LR', 'CESM2', 'ACCESS-ESM1-5', 'IPSL-CM6A-LR']
# simu_list  = ['picontrol', 'historical', 'ssp126', 'ssp585']
simu_list  = ['historical', 'ssp126', 'ssp585']
# predname_list = ['dissic', 'talk', 'tos', 'sos', 'siconc', 'sfcWind', 'po4', 'epc100', 'mlotst']
predname_list = ['dissic', 'talk', 'tos', 'sos', 'sfcWind', 'po4', 'epc100', 'mlotst']
#predname_list = ['dissic', 'talk']
model = 'ACCESS-ESM1-5'


2022-11-08 11:33:14.345192
CPU times: user 283 µs, sys: 40 µs, total: 323 µs
Wall time: 232 µs


## Load and prepare data2plot

In [ ]:
%%time
print(datetime.datetime.now())
print('Variances of the detrended and deseasonalized predictors: prepare and save data2plot')

print('Model: %s'%model)
data2save = {}

for predname in predname_list: 
    
    print('> predname: %s' %predname)

    data2save[predname] = {}
    
    for simu in simu_list: 
        
        print('>> simu: %s' %simu)
        
        data2save[predname][simu] = {}

        #---------------
        # Read datas
        #---------------
    
        dirname = dircmip6 + model + '/' + simu + '/'
        fname = predname + '_' + model + '_' + simu + '*.nc'
        dataset = xr.open_mfdataset(dirname + fname, use_cftime=True)
        zwpred = dataset[predname]

        zwX = zwpred['lon'].values
        zwY = zwpred['lat'].values
        zwZ = zwpred.values

        X, Y = np.meshgrid(zwX, zwY)
        
        if len(zwZ.shape)== 4 : nt, nz, ny, nx = zwZ.shape
        elif len(zwZ.shape)==3: nt,     ny, nx = zwZ.shape
        
        #---------------
        # Compute variance of detrended and deseasonalized signal
        #---------------

        Z = np.zeros((ny, nx))
        if len(zwZ.shape)== 4 : ground = np.isnan(zwZ[0, 0])    
        elif len(zwZ.shape)==3: ground = np.isnan(zwZ[0])    
        for jj in np.arange(ny): 
            for ii in np.arange(nx): 
                if not ground[jj, ii]: 
                    if len(zwZ.shape)== 4 : zw = zwZ[:, 0, jj, ii]
                    elif len(zwZ.shape)==3: zw = zwZ[:,    jj, ii]
                    # trend
                    t = np.linspace(0, nt - 1, num=nt)
                    trd = p_smooth_spline(t, zw)
                    # dtrd, dseas
                    zw_dtrd_dseas, trdseas, seas = dtrd_dseas(zw, trd)
                    Z[jj, ii] = np.var(zw_dtrd_dseas)                
                else : Z[jj, ii] = np.nan
            #
        #

        #---------------
        # Save data
        #---------------

        data2save[predname][simu]['X'] = X
        data2save[predname][simu]['Y'] = Y
        data2save[predname][simu]['Z'] = Z
    #
#

savedfile = dirout + 'data2plot-variances-of-dtrd-deseas-predictors-'+model+'.pckl'
if os.path.isfile(savedfile): os.remove(savedfile)
with open(savedfile, 'wb') as f1:pickle.dump(data2save, f1)
print("File saved: "+savedfile)        

# ca. 9h

2022-11-08 11:35:01.191731
Variances of the detrended and deseasonalized predictors: prepare and save data2plot
Model: ACCESS-ESM1-5
> predname: dissic
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> predname: talk
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> predname: tos
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> predname: sos
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> predname: sfcWind
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> predname: po4
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> predname: epc100
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> predname: mlotst
>> simu: historical


/opt/conda/lib/python3.7/site-packages/scipy/interpolate/fitpack2.py:232: UserWarning: 
The maximal number of iterations maxit (set to 20 by the program)
allowed for finding a smoothing spline with fp=s has been reached: s
too small.
There is an approximation returned but the corresponding weighted sum
of squared residuals does not satisfy the condition abs(fp-s)/s < tol.
  warnings.warn(message)


## Load and prepare salinity normalized DIC and ALK variance (dtrd and deseas)

In [ ]:
%%time
print(datetime.datetime.now())
print('Variances of the detrended and deseasonalized predictors: prepare and save salinity normalized DIC and ALK variance (dtrd and deseas)')

print('Model: %s'%model)
data2save = {}

for predname in ['dissic', 'talk']: 
    
    print('> predname: %s' %predname)

    data2save[predname] = {}
    
    for simu in simu_list: 
        
        print('>> simu: %s' %simu)
        
        data2save[predname][simu] = {}

        #---------------
        # Read datas
        #---------------
    
        dirname = dircmip6 + model + '/' + simu + '/'
        fname = 'sos' + '_' + model + '_' + simu + '*.nc'
        dataset = xr.open_mfdataset(dirname + fname)
        zwsos = dataset['sos']
        zwS0 = zwsos.mean(axis=0).values
        zwS  = zwsos.values

        dirname = dircmip6 + model + '/' + simu + '/'
        fname = predname + '_' + model + '_' + simu + '*.nc'
        dataset = xr.open_mfdataset(dirname + fname)
        zwpred = dataset[predname]

        zwX = zwpred['lon'].values
        zwY = zwpred['lat'].values
        zwZ = zwpred.values
        
        X, Y = np.meshgrid(zwX, zwY)
        
        if len(zwZ.shape)== 4 : nt, nz, ny, nx = zwZ.shape
        elif len(zwZ.shape)==3: nt,     ny, nx = zwZ.shape
        
        #---------------
        # Compute variance of detrended and deseasonalized signal
        #---------------

        Z = np.zeros((ny, nx))
        if len(zwZ.shape)== 4 : ground = np.isnan(zwZ[0, 0])    
        elif len(zwZ.shape)==3: ground = np.isnan(zwZ[0])    
        for jj in np.arange(ny): 
            for ii in np.arange(nx): 
                if not ground[jj, ii]: 
                    if len(zwZ.shape)== 4 : zw = zwZ[:, 0, jj, ii] * zwS0[np.newaxis, jj, ii] / zwS[:, jj, ii]
                    elif len(zwZ.shape)==3: zw = zwZ[:,    jj, ii] * zwS0[np.newaxis, jj, ii] / zwS[:, jj, ii]
                    # trend
                    t = np.linspace(0, nt - 1, num=nt)
                    trd = p_smooth_spline(t, zw)
                    # dtrd, dseas
                    zw_dtrd_dseas, trdseas, seas = dtrd_dseas(zw, trd)
                    Z[jj, ii] = np.var(zw_dtrd_dseas)                
                else : Z[jj, ii] = np.nan
            #
        #

        #---------------
        # Save data
        #---------------

        data2save[predname][simu]['X'] = X
        data2save[predname][simu]['Y'] = Y
        data2save[predname][simu]['Z'] = Z
    #
#

savedfile = dirout + 'data2plot-variances-of-dtrd-deseas-dic-alk-sal-normalized-'+model+'.pckl'
if os.path.isfile(savedfile): os.remove(savedfile)
with open(savedfile, 'wb') as f1:pickle.dump(data2save, f1)
print("File saved: "+savedfile)        

# ca. 6min

# Variances of the detrended and deseasonalized FGCO2

## Inputs

In [6]:
%%time
print(datetime.datetime.now())

model_list = ['NorESM2-LM', 'MPI-ESM1-2-LR', 'CESM2', 'ACCESS-ESM1-5', 'IPSL-CM6A-LR']
# simu_list  = ['picontrol', 'historical', 'ssp126', 'ssp585']
simu_list  = ['historical', 'ssp126', 'ssp585']
predname = 'fgco2'


2022-09-21 14:37:29.927732
CPU times: user 471 µs, sys: 46 µs, total: 517 µs
Wall time: 419 µs


## Load and prepare data2plot

In [ ]:
%%time
print(datetime.datetime.now())
print('Variances of the detrended and deseasonalized FGCO2: prepare and save data2plot')

data2save = {}

for model in model_list: 
    
    print('> model: %s' %model)

    data2save[model] = {}
    
    for simu in simu_list: 
        
        print('>> simu: %s' %simu)
        
        data2save[model][simu] = {}

        #---------------
        # Read datas
        #---------------
    
        dirname = dircmip6 + model + '/' + simu + '/'
        fname = predname + '_' + model + '_' + simu + '*.nc'
        dataset = xr.open_mfdataset(dirname + fname)
        zwpred = dataset[predname]

        zwX = zwpred['lon'].values
        zwY = zwpred['lat'].values
        zwZ = zwpred.values

        X, Y = np.meshgrid(zwX, zwY)
        
        if len(zwZ.shape)== 4 : nt, nz, ny, nx = zwZ.shape
        elif len(zwZ.shape)==3: nt,     ny, nx = zwZ.shape
        
        #---------------
        # Compute variance of detrended and deseasonalized signal
        #---------------

        Z = np.zeros((ny, nx))
        if len(zwZ.shape)== 4 : ground = np.isnan(zwZ[0, 0])    
        elif len(zwZ.shape)==3: ground = np.isnan(zwZ[0])    
        for jj in np.arange(ny): 
            for ii in np.arange(nx): 
                if not ground[jj, ii]: 
                    if len(zwZ.shape)== 4 : zw = zwZ[:, 0, jj, ii]
                    elif len(zwZ.shape)==3: zw = zwZ[:,    jj, ii]
                    # trend
                    t = np.linspace(0, nt - 1, num=nt)
                    trd = p_smooth_spline(t, zw)
                    # dtrd, dseas
                    zw_dtrd_dseas, trdseas, seas = dtrd_dseas(zw, trd)
                    Z[jj, ii] = np.var(zw_dtrd_dseas)                
                else : Z[jj, ii] = np.nan
            #
        #

        #---------------
        # Save data
        #---------------

        data2save[model][simu]['X'] = X
        data2save[model][simu]['Y'] = Y
        data2save[model][simu]['Z'] = Z
    #
#

savedfile = dirout + 'data2plot-variances-of-dtrd-deseas-fgco2.pckl'
if os.path.isfile(savedfile): os.remove(savedfile)
with open(savedfile, 'wb') as f1:pickle.dump(data2save, f1)
print("File saved: "+savedfile)        

# ca. 9h

2022-09-21 14:37:32.174816
Variances of the detrended and deseasonalized FGCO2: prepare and save data2plot
> model: NorESM2-LM
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> model: MPI-ESM1-2-LR
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> model: CESM2
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> model: ACCESS-ESM1-5
>> simu: historical
>> simu: ssp126
>> simu: ssp585
> model: IPSL-CM6A-LR
>> simu: historical


# Variances of the detrended and deseasonalized DIC, ALK and DICsn, ALKsn

## Inputs

In [5]:
%%time
print(datetime.datetime.now())

model_list = ['NorESM2-LM', 'MPI-ESM1-2-LR', 'CESM2', 'ACCESS-ESM1-5', 'IPSL-CM6A-LR']
#model_list = ['IPSL-CM6A-LR']
# simu_list  = ['picontrol', 'historical', 'ssp126', 'ssp585']
simu_list  = ['historical', 'ssp126', 'ssp585']
predname_list = ['dissic', 'talk']

2022-09-13 17:27:39.358085
CPU times: user 101 µs, sys: 15 µs, total: 116 µs
Wall time: 84.6 µs


## Load and prepare data2plot

In [7]:
%%time
print(datetime.datetime.now())
print('Variances of the detrended and deseasonalized DIC, ALK and DICsn, ALKsn: prepare and save data2plot')

for model in model_list: 
    
    print('> Model: %s'%model)
    data2save = {}

    for predname in predname_list: 

        print('>> predname: %s' %predname)

        data2save[predname] = {}
        data2save[predname+'_sn'] = {}

        for simu in simu_list: 

            print('>>> simu: %s' %simu)

            data2save[predname][simu] = {}
            data2save[predname+'_sn'][simu] = {}

            #---------------
            # Read datas
            #---------------

            print('>>>> Read datas')
            
            #_______________
            # DIC or ALK

            dirname = dircmip6 + model + '/' + simu + '/'
            fname = predname + '_' + model + '_' + simu + '*.nc'
            dataset = xr.open_mfdataset(dirname + fname, use_cftime=True)
            print('File loaded: %s' %fname)
            zwpred = dataset[predname].sel(time=slice('1850', '2100'))

            zwX = zwpred['lon'].values
            zwY = zwpred['lat'].values
            zwZ = zwpred.values

            X, Y = np.meshgrid(zwX, zwY)

            if len(zwZ.shape)== 4 : nt, nz, ny, nx = zwZ.shape
            elif len(zwZ.shape)==3: nt,     ny, nx = zwZ.shape

            #_______________
            # Salinity

            fname = 'sos' + '_' + model + '_' + simu + '*.nc'
            dataset = xr.open_mfdataset(dirname + fname, use_cftime=True)
            zwsos = dataset['sos'].sel(time=slice('1850', '2100'))
            zwS0 = zwsos.mean(axis=0).values
            zwS  = zwsos.values
            print('File loaded: %s' %fname)

            #---------------
            # Compute variance of detrended and deseasonalized signal
            #---------------

            print('>>>> Compute variances')
            
            Z = np.zeros((ny, nx))
            Zs = np.zeros((ny, nx))
            if len(zwZ.shape)== 4 : ground = np.isnan(zwZ[0, 0])    
            elif len(zwZ.shape)==3: ground = np.isnan(zwZ[0])    
            for jj in np.arange(ny): 
                for ii in np.arange(nx): 

                    if not ground[jj, ii]: 

                        if len(zwZ.shape)== 4 : zw = zwZ[:, 0, jj, ii]
                        elif len(zwZ.shape)==3: zw = zwZ[:,    jj, ii]
                        # trend
                        t = np.linspace(0, nt - 1, num=nt)
                        trd = p_smooth_spline(t, zw)
                        # dtrd, dseas
                        zw_dtrd_dseas, trdseas, seas = dtrd_dseas(zw, trd)
                        Z[jj, ii] = np.var(zw_dtrd_dseas)    

                        #_______________
                        # Salinity normalized
                        if len(zwZ.shape)== 4 : zw = zwZ[:, 0, jj, ii] * zwS0[np.newaxis, jj, ii] / zwS[:, jj, ii]
                        elif len(zwZ.shape)==3: zw = zwZ[:,    jj, ii] * zwS0[np.newaxis, jj, ii] / zwS[:, jj, ii]
                        # trend
                        t = np.linspace(0, nt - 1, num=nt)
                        trd = p_smooth_spline(t, zw)
                        # dtrd, dseas
                        zw_dtrd_dseas, trdseas, seas = dtrd_dseas(zw, trd)
                        Zs[jj, ii] = np.var(zw_dtrd_dseas)            

                    else : 
                        Z[jj, ii] = np.nan
                        Zs[jj, ii] = np.nan

                #
            #

            #---------------
            # Save data
            #---------------

            data2save[predname][simu]['X'] = X
            data2save[predname][simu]['Y'] = Y
            data2save[predname][simu]['Z'] = Z

            data2save[predname+'_sn'][simu]['X'] = X
            data2save[predname+'_sn'][simu]['Y'] = Y
            data2save[predname+'_sn'][simu]['Z'] = Zs
        #
    #

    #---------------
    # Save file
    #---------------
    print('> Save file')
    savedfile = dirout + 'data2plot-variances-of-dtrd-deseas-dic-alk-dicsn-alksn-'+model+'.pckl'
    if os.path.isfile(savedfile): os.remove(savedfile)
    with open(savedfile, 'wb') as f1:pickle.dump(data2save, f1)
    print("File saved: "+savedfile)        

# ca. 56m

2022-09-07 13:28:50.824521
Variances of the detrended and deseasonalized DIC, ALK and DICsn, ALKsn: prepare and save data2plot
> Model: NorESM2-LM
>> predname: dissic
>>> simu: historical
>>>> Read datas
File loaded: dissic_NorESM2-LM_historical*.nc
File loaded: sos_NorESM2-LM_historical*.nc
>>>> Compute variances


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


>>> simu: ssp126
>>>> Read datas
File loaded: dissic_NorESM2-LM_ssp126*.nc
File loaded: sos_NorESM2-LM_ssp126*.nc
>>>> Compute variances
>>> simu: ssp585
>>>> Read datas
File loaded: dissic_NorESM2-LM_ssp585*.nc
File loaded: sos_NorESM2-LM_ssp585*.nc
>>>> Compute variances
>> predname: talk
>>> simu: historical
>>>> Read datas
File loaded: talk_NorESM2-LM_historical*.nc
File loaded: sos_NorESM2-LM_historical*.nc
>>>> Compute variances


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


>>> simu: ssp126
>>>> Read datas
File loaded: talk_NorESM2-LM_ssp126*.nc
File loaded: sos_NorESM2-LM_ssp126*.nc
>>>> Compute variances
>>> simu: ssp585
>>>> Read datas
File loaded: talk_NorESM2-LM_ssp585*.nc
File loaded: sos_NorESM2-LM_ssp585*.nc
>>>> Compute variances
> Save file
File saved: dimred-220513-predictors-variances/data2plot-variances-of-dtrd-deseas-dic-alk-dicsn-alksn-NorESM2-LM.pckl
> Model: MPI-ESM1-2-LR
>> predname: dissic
>>> simu: historical
>>>> Read datas
File loaded: dissic_MPI-ESM1-2-LR_historical*.nc
File loaded: sos_MPI-ESM1-2-LR_historical*.nc
>>>> Compute variances
>>> simu: ssp126
>>>> Read datas
File loaded: dissic_MPI-ESM1-2-LR_ssp126*.nc
File loaded: sos_MPI-ESM1-2-LR_ssp126*.nc
>>>> Compute variances
>>> simu: ssp585
>>>> Read datas
File loaded: dissic_MPI-ESM1-2-LR_ssp585*.nc
File loaded: sos_MPI-ESM1-2-LR_ssp585*.nc
>>>> Compute variances
>> predname: talk
>>> simu: historical
>>>> Read datas
File loaded: talk_MPI-ESM1-2-LR_historical*.nc
File loaded: s

...

>>> simu: ssp126
>>>> Read datas
File loaded: talk_ACCESS-ESM1-5_ssp126*.nc
File loaded: sos_ACCESS-ESM1-5_ssp126*.nc
>>>> Compute variances


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


>>> simu: ssp585
>>>> Read datas
File loaded: talk_ACCESS-ESM1-5_ssp585*.nc
File loaded: sos_ACCESS-ESM1-5_ssp585*.nc
>>>> Compute variances


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: Mean of empty slice
  # Remove the CWD from sys.path while we load stuff.


> Save file
File saved: dimred-220513-predictors-variances/data2plot-variances-of-dtrd-deseas-dic-alk-dicsn-alksn-ACCESS-ESM1-5.pckl
> Model: IPSL-CM6A-LR
>> predname: dissic
>>> simu: historical
>>>> Read datas
File loaded: dissic_IPSL-CM6A-LR_historical*.nc
File loaded: sos_IPSL-CM6A-LR_historical*.nc
>>>> Compute variances
>>> simu: ssp126
>>>> Read datas
File loaded: dissic_IPSL-CM6A-LR_ssp126*.nc
File loaded: sos_IPSL-CM6A-LR_ssp126*.nc
>>>> Compute variances
>>> simu: ssp585
>>>> Read datas
File loaded: dissic_IPSL-CM6A-LR_ssp585*.nc
File loaded: sos_IPSL-CM6A-LR_ssp585*.nc
>>>> Compute variances
>> predname: talk
>>> simu: historical
>>>> Read datas
File loaded: talk_IPSL-CM6A-LR_historical*.nc
File loaded: sos_IPSL-CM6A-LR_historical*.nc
>>>> Compute variances
>>> simu: ssp126
>>>> Read datas
File loaded: talk_IPSL-CM6A-LR_ssp126*.nc
File loaded: sos_IPSL-CM6A-LR_ssp126*.nc
>>>> Compute variances
>>> simu: ssp585
>>>> Read datas
File loaded: talk_IPSL-CM6A-LR_ssp585*.nc
File lo

# Time average maps of predictors

## Inputs

In [8]:
%%time
print(datetime.datetime.now())

# model_list = ['NorESM2-LM', 'MPI-ESM1-2-LR', 'CNRM-ESM2-1', 'CESM2', 'ACCESS-ESM1-5', 'IPSL-CM6A-LR']
# simu_list  = ['picontrol', 'historical', 'ssp126', 'ssp585']
simu_list  = ['historical', 'ssp126', 'ssp585']
predname_list = ['dissic', 'talk', 'tos', 'sos', 'siconc', 'sfcWind', 'po4', 'epc100', 'mlotst']
#predname_list = ['dissic', 'talk']
model = 'NorESM2-LM'


2022-08-12 17:08:33.791141
CPU times: user 0 ns, sys: 219 µs, total: 219 µs
Wall time: 131 µs


## Load and prepare data2plot

In [9]:
%%time
print(datetime.datetime.now())
print('Time average maps of predictors: prepare and save data2plot')

print('Model: %s'%model)
data2save = {}

for predname in predname_list: 
    
    print('> predname: %s' %predname)

    data2save[predname] = {}
    
    for simu in simu_list: 
        
        print('>> simu: %s' %simu)
        
        data2save[predname][simu] = {}

        #---------------
        # Read datas
        #---------------
    
        dirname = dircmip6 + model + '/' + simu + '/'
        fname = predname + '_' + model + '_' + simu + '*.nc'
        dataset = xr.open_mfdataset(dirname + fname)
        zwpred = dataset[predname]

        zwX = zwpred['lon'].values
        zwY = zwpred['lat'].values
        zwZ = zwpred.values

        X, Y = np.meshgrid(zwX, zwY)
        
        if len(zwZ.shape)== 4 : nt, nz, ny, nx = zwZ.shape
        elif len(zwZ.shape)==3: nt,     ny, nx = zwZ.shape
        
        #---------------
        # Compute time mean
        #---------------

        Z = np.nanmean(zwZ, axis = 0).squeeze()

        #---------------
        # Save data
        #---------------

        data2save[predname][simu]['X'] = X
        data2save[predname][simu]['Y'] = Y
        data2save[predname][simu]['Z'] = Z
    #
#

savedfile = dirout + 'data2plot-predictors-time-avg-'+model+'.pckl'
if os.path.isfile(savedfile): os.remove(savedfile)
with open(savedfile, 'wb') as f1:pickle.dump(data2save, f1)
print("File saved: "+savedfile)        

# ca. 1m

2022-08-12 17:09:00.545226
Time average maps of predictors: prepare and save data2plot
Model: NorESM2-LM
> predname: dissic
>> simu: historical


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: Mean of empty slice


>> simu: ssp126


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: Mean of empty slice


>> simu: ssp585


...

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: Mean of empty slice


>> simu: ssp126


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: Mean of empty slice


>> simu: ssp585
File saved: dimred-220513-predictors-variances/data2plot-predictors-time-avg-NorESM2-LM.pckl
CPU times: user 28.1 s, sys: 17.2 s, total: 45.2 s
Wall time: 1min 21s


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:41: RuntimeWarning: Mean of empty slice


## Fig

In [7]:
%%time
print(datetime.datetime.now())
print('Time average maps of predictors: fig')

print('Model: %s'%model)
savedfile = dirout + 'data2plot-predictors-time-avg-'+model+'.pckl'
with open(savedfile, 'rb') as f1: data2plot = pickle.load(f1)
print("File loaded: "+savedfile)        

#-----------------
# FIGURE PARAM
#-----------------

cm2in = 1/2.54 # converting factor cm to inch 
xfsize, yfsize = 4, 3
nrow = len(predname_list)
ncol = len(simu_list)
ccrsproj = ccrs.Robinson(central_longitude=-155)
fig, ax = plt.subplots(nrow, ncol, figsize=(xfsize*ncol,yfsize*nrow), 
                       subplot_kw=dict(projection=ccrsproj),
                       squeeze = False)


#-----------------
# CREATE CUSTOM CMAP
#-----------------

from matplotlib.colors import ListedColormap
from matplotlib import cm
cmap = cm.get_cmap('viridis', 256)
newcolors = cmap(np.linspace(0, 1, 10+2))
cmap = ListedColormap(newcolors[1:-1])
cmap.set_under(color=newcolors[0])
cmap.set_over(color=newcolors[-1])

#-----------------
# KEYWORDS DICT
#-----------------

# kwmap = {'vmin':0, 'vmax':1, 'cmap':cmap, \
#          'transform':ccrs.PlateCarree() }
kwmap = {'cmap':cmap, \
         'transform':ccrs.PlateCarree() }
vmax = {
    'dissic' : 2.2,   
    'talk'   : 2.5,    
    'tos'    : 30,   
    'sos'    : 37,   
    'siconc' : 70,   
    'sfcWind': 12,   
    'po4'    : 0.0015,    
    'epc100' : 1e-7,    
    'mlotst' : 150   
}

vmin = {
    'dissic' : 1.9,    
    'talk'   : 2.1,    
    'tos'    : 0,    
    'sos'    : 32,    
    'siconc' : 10,    
    'sfcWind': 3,    
    'po4'    : 0,    
    'epc100' : 2e-8,    
    'mlotst' : 20    
}

#---------------------
# Plot
#---------------------

irow = 0
pcm = np.zeros_like(ax)
for predname in predname_list: 
    icol = 0
    for simu in simu_list: 

        zax = ax[irow, icol]
        X = data2plot[predname][simu]['X']
        Y = data2plot[predname][simu]['Y']
        Z = data2plot[predname][simu]['Z']
        # pcm = zax.pcolormesh(X, Y, Z/np.nanmean(Z), **kwmap)    
        # zax.set_title('%s, %s\n %.1e' %(model, simu.upper(), np.nanmean(Z)))
        zwpcm = zax.pcolormesh(X, Y, Z, vmin=vmin[predname], vmax=vmax[predname], **kwmap)    
        zax.set_title('%s, %s' %(predname, simu.upper()))
        zax.coastlines()
        # gl = zax.gridlines(crs=ccrs.PlateCarree(), linewidth=1)
        pcm[irow, icol] = zwpcm
        icol+=1

    #
    irow+=1
#

fig.tight_layout()

#---------------------
# Move axis
#---------------------

dy = .3*ax[0, 0].get_position().height
for iaxrow, axrow in enumerate(ax[1:]): 
    for zax in axrow: 
        
        zw = zax.get_position()
        ny0 = zw.y0 + (iaxrow+1)*dy
        zax.set_position([zw.x0, ny0, zw.width, zw.height])
    #
#
        


#---------------------
# Colorbar
#---------------------

for ipredname, vpredname in enumerate(predname_list): 
    zax = ax[ipredname, 0]
    zpcm = pcm[ipredname, 0]
    zw = zax.get_position()
    nx0 = zw.x0 - 0.1*zw.width
    ny0 = zw.y0
    nh  = zw.height
    nw  = 0.1*nh
    cax = fig.add_axes([nx0, ny0, nw, nh])
    cbar = fig.colorbar(zpcm, cax=cax, orientation='vertical', \
                        ticklocation='left', extend='both')
    cbar.set_label('')
#
#---------------------
# Save figure
#---------------------

fignam = 'maps-predictors-time-avg-'+model+'.png'
fig.savefig(dirout+fignam, bbox_inches='tight')
print('Figure saved: '+fignam)

# ca. 40s

2022-08-12 17:07:58.165428
Time average maps of predictors: fig
Model: NorESM2-LM


FileNotFoundError: [Errno 2] No such file or directory: 'dimred-220513-predictors-variances/data2plot-predictors-time-avg-NorESM2-LM.pckl'